<a href="https://colab.research.google.com/github/Daegil2/practice/blob/main/job_catboost%2Bcv5fold%2Bthreshold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google import colab
colab.drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 63 kB/s 


In [3]:
import os
import sys
import platform
import random
import math
from typing import List ,Dict, Tuple

import pandas as pd
import numpy as np
 
import sklearn 
from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import f1_score 

from catboost import Pool,CatBoostClassifier

In [4]:
D_PATH ='/content/drive/MyDrive/corab/job/'
S_PATH ='/content/drive/MyDrive/corab/job/'


In [5]:
train =pd.read_csv('/content/drive/MyDrive/corab/job/train.csv')
test=pd.read_csv('/content/drive/MyDrive/corab/job/test.csv')

In [6]:
code_d = pd.read_csv(f'{D_PATH}속성_D_코드.csv').iloc[:,:-1]
code_h = pd.read_csv(f'{D_PATH}속성_H_코드.csv')
code_l = pd.read_csv(f'{D_PATH}속성_L_코드.csv')


train.shape,test.shape

((501951, 35), (46404, 34))

In [7]:
#속성 코드 데이터 컬럼 변경
code_d.columns= ["attribute_d","attribute_d_d","attribute_d_s","attribute_d_m","attribute_d_l"]
code_h.columns= ["attribute_h","attribute_h_p"]
code_l.columns= ["attribute_l","attribute_l_d","attribute_l_s","attribute_l_m","attribute_l_l"]

In [8]:
#속성 코드 데이터 merge

def merge_codes(df:pd.DataFrame, df_code:pd.DataFrame, col:str)->pd.DataFrame:
  df =df.copy()
  df_code =df_code
  df_code= df_code.add_prefix(f'{col}_')
  df_code.columns.values[0] =col
  return pd.merge(df, df_code, how='left', on=col)

In [9]:
#데이터 전처리
def preprocess_data(
                    df:pd.DataFrame,is_train:bool = True, cols_merge:List[Tuple[str,pd.DataFrame]] = []  , cols_equi:List[Tuple[str,str]]= [] ,
                    cols_drop:List[str] = ["id","person_prefer_f","person_prefer_g" ,"contents_open_dt"]
                    )->Tuple[pd.DataFrame,np.ndarray]:
    df = df.copy()
    
    y_data=None
    if is_train:
      y_data=df['target'].to_numpy()
      df =df.drop(columns='target')
    
    for col, df_code in cols_merge:
      df =merge_codes(df, df_code, col)
    
    cols = df.select_dtypes(bool).columns.tolist()
    df[cols] =df[cols].astype(int)

    for col1, col2 in cols_equi:
      df[f'{col}_{col2}']= (df[col1]==df[col2]).astype(int)

    df =df.drop(columns =cols_drop)
    return (df, y_data)

In [10]:
#전처리 컬럼명 정의
# 소분류 중분류 대분류 속성코드 merge 컬럼명 및 데이터 프레임 리스트
cols_merge = [
              ("person_prefer_d_1" , code_d),
              ("person_prefer_d_2" , code_d),
              ("person_prefer_d_3" , code_d),
              ("contents_attribute_d" , code_d),
              ("person_prefer_h_1" , code_h),
              ("person_prefer_h_2" , code_h),
              ("person_prefer_h_3" , code_h),
              ("contents_attribute_h" , code_h),
              ("contents_attribute_l" , code_l),
]

# 회원 속성과 콘텐츠 속성의 동일한 코드 여부에 대한 컬럼명 리스트
cols_equi = [

    ("contents_attribute_c","person_prefer_c"),
    ("contents_attribute_e","person_prefer_e"),

    ("person_prefer_d_2_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_2_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_2_attribute_d_l" , "contents_attribute_d_attribute_d_l"),
    ("person_prefer_d_3_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_3_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_3_attribute_d_l" , "contents_attribute_d_attribute_d_l"),

    ("person_prefer_h_1_attribute_h_p" , "contents_attribute_h_attribute_h_p"),
    ("person_prefer_h_2_attribute_h_p" , "contents_attribute_h_attribute_h_p"),
    ("person_prefer_h_3_attribute_h_p" , "contents_attribute_h_attribute_h_p"),

]

# 학습에 필요없는 컬럼 리스트
cols_drop = ["id","person_prefer_f","person_prefer_g" ,"contents_open_dt", "contents_rn", ]

In [11]:
#학습 추론전처리
x_train, y_train= preprocess_data(train, cols_merge =cols_merge, cols_equi =cols_equi, cols_drop=cols_drop)
x_test, _ =preprocess_data(test,is_train =False, cols_merge=cols_merge, cols_equi=cols_equi, cols_drop=cols_drop)
x_train.shape, y_train.shape, x_test.shape

((501951, 59), (501951,), (46404, 59))

In [12]:
#범주형 컬럼 리스트
cat_features= x_train.columns[x_train.nunique()>2].tolist()

In [13]:
#학습 파라미터
is_holdout=False
n_splits=5
iterations=3000
patience=50

cv=KFold(n_splits=n_splits, shuffle= True, random_state =42)


In [14]:
scores=[]
models=[]

models=[]

for tri,vai in cv.split(x_train):
  print('='*50)
  pred=[]
  
  model = CatBoostClassifier(iterations=iterations, random_state=42, task_type='GPU', eval_metric='F1', cat_features =cat_features, one_hot_max_size=4)
  model.fit(x_train.iloc[tri], y_train[tri],
            eval_set=[(x_train.iloc[vai], y_train[vai])],
            early_stopping_rounds=patience,
            verbose =100
            )
  models.append(model)
  scores.append(model.get_best_score()['validation']['F1'])

  if is_holdout:
    break

Learning rate set to 0.027144
0:	learn: 0.5887153	test: 0.5965150	best: 0.5965150 (0)	total: 332ms	remaining: 16m 36s
100:	learn: 0.6423237	test: 0.6506937	best: 0.6507186 (99)	total: 35.1s	remaining: 16m 47s
200:	learn: 0.6541098	test: 0.6720348	best: 0.6720348 (200)	total: 1m 19s	remaining: 18m 22s
300:	learn: 0.6589402	test: 0.6804977	best: 0.6805884 (296)	total: 1m 59s	remaining: 17m 47s
400:	learn: 0.6626619	test: 0.6829666	best: 0.6831206 (397)	total: 2m 40s	remaining: 17m 17s
500:	learn: 0.6655673	test: 0.6844292	best: 0.6846300 (497)	total: 3m 14s	remaining: 16m 10s
bestTest = 0.6846592779
bestIteration = 502
Shrink model to first 503 iterations.
Learning rate set to 0.027144
0:	learn: 0.6098259	test: 0.6122721	best: 0.6122721 (0)	total: 396ms	remaining: 19m 46s
100:	learn: 0.6439252	test: 0.6566550	best: 0.6567002 (99)	total: 34.9s	remaining: 16m 41s
200:	learn: 0.6534293	test: 0.6718295	best: 0.6719345 (198)	total: 1m 6s	remaining: 15m 28s
300:	learn: 0.6591129	test: 0.680513

In [14]:
print(scores)
print(np.mean(scores))

In [16]:
threshold= 0.4

In [17]:
pred_list= []
scores=[]

for i,(tri, vai) in enumerate(cv.split(x_train)):
  pred= models[i].predict_proba(x_train.iloc[vai])[:,1]
  pred = np.where(pred >= threshold , 1, 0)
  score =f1_score(y_train[vai], pred)
  scores.append(score)
  pred =models[i].predict_proba(x_test)[:,1]
  pred_list.append(pred)

print(scores)
print(np.mean(scores))



[0.7114741684188478, 0.7101443448192675, 0.7092062364827788, 0.7087434576216173, 0.7086264950227057]
0.7096389404730433


In [18]:
pred =np.mean(pred_list, axis=0)
pred= np.where(pred>=threshold, 1,0)


In [21]:
sample_submission = pd.read_csv(f'{D_PATH}sample_submission.csv')
sample_submission['target'] = pred
sample_submission

,id,target
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
46399,46399,1
46400,46400,1
46401,46401,1
46402,46402,1
